In [3]:
import csv
import os
import re
from datetime import datetime
from pathlib import Path

class MeteoCSVConverter:
    def __init__(self):
        self.months = {
            'janvier': 1, 'january': 1,
            'février': 2, 'fevrier': 2, 'february': 2,
            'mars': 3, 'march': 3,
            'avril': 4, 'april': 4,
            'mai': 5, 'may': 5,
            'juin': 6, 'june': 6,
            'juillet': 7, 'july': 7,
            'aoыt': 8, 'août': 8, 'august': 8,
            'septembre': 9, 'september': 9,
            'octobre': 10, 'october': 10,
            'novembre': 11, 'november': 11,
            'décembre': 12, 'decembre': 12, 'dйcembre': 12, 'december': 12
        }
        
    def extract_year_from_filename(self, filename):
        """Extrait l'année du nom de fichier"""
        # Convertir en string si c'est un Path object
        if hasattr(filename, 'name'):
            filename = filename.name
        elif not isinstance(filename, str):
            filename = str(filename)
            
        match = re.search(r'(\d{4})', filename)
        if match:
            return int(match.group(1))
        
        # Si pas d'année trouvée, essayer d'extraire de patterns comme "95" pour 1995
        match = re.search(r'(\d{2})', filename)
        if match:
            year_short = int(match.group(1))
            # Supposer que 00-30 = 2000-2030, 31-99 = 1931-1999
            if year_short <= 30:
                return 2000 + year_short
            else:
                return 1900 + year_short
        
        return 2023  # Année par défaut
    
    def parse_csv_file(self, file_path):
        """Parse un fichier CSV et retourne les données structurées"""
        year = self.extract_year_from_filename(file_path)
        
        # S'assurer que file_path est un string pour open()
        if hasattr(file_path, '__fspath__'):  # Path object
            file_path_str = str(file_path)
        else:
            file_path_str = file_path
        
        with open(file_path_str, 'r', encoding='utf-8', errors='ignore') as file:
            content = file.read()
        
        # Nettoyer le contenu
        lines = content.strip().split('\n')
        reader = csv.reader(lines, delimiter=';')
        
        try:
            headers = next(reader)
        except StopIteration:
            return []
        
        print(f"Traitement de {file_path} (année {year})")
        print(f"En-têtes: {headers}")
        
        # Dictionnaire pour regrouper les données par date
        daily_data = {}
        
        for row in reader:
            if not row or not row[0] or not row[0].strip():
                continue
                
            try:
                day = int(row[0].strip())
            except ValueError:
                continue
            
            # Traiter chaque colonne (mois)
            for col_index, cell_value in enumerate(row[1:], 1):
                if col_index >= len(headers):
                    break
                    
                header = headers[col_index].strip().lower()
                
                if not cell_value or not cell_value.strip():
                    continue
                
                # Vérifier si c'est une colonne de neige
                is_snow_column = 'neige' in header
                
                # Extraire le mois de l'en-tête
                month_num = None
                for month_name, month_num_val in self.months.items():
                    if month_name in header:
                        month_num = month_num_val
                        break
                
                if month_num is None:
                    continue
                
                try:
                    # Remplacer virgule par point pour les décimales
                    value = float(cell_value.replace(',', '.'))
                    
                    # Créer la date
                    date_str = f"{year}-{month_num:02d}-{day:02d}"
                    
                    # Vérifier si la date est valide
                    try:
                        datetime.strptime(date_str, '%Y-%m-%d')
                    except ValueError:
                        continue
                    
                    # Initialiser l'entrée pour cette date si elle n'existe pas
                    if date_str not in daily_data:
                        daily_data[date_str] = {
                            'date': date_str,
                            'quantite_mm': 0.0,
                            'neige_cm': 0.0,
                            'commentaire': ''
                        }
                    
                    # Ajouter les données selon le type
                    if is_snow_column:
                        daily_data[date_str]['neige_cm'] = value
                    else:
                        daily_data[date_str]['quantite_mm'] = value
                    
                except ValueError:
                    continue
        
        # Convertir le dictionnaire en liste
        return list(daily_data.values())
    
    def process_directory(self, directory_path, output_file='precipitations_import.sql'):
        """Traite tous les fichiers CSV d'un répertoire"""
        directory = Path(directory_path)
        
        if not directory.exists():
            print(f"Erreur: Le répertoire {directory_path} n'existe pas")
            return
        
        # Dictionnaire pour regrouper toutes les données par date
        all_daily_data = {}
        csv_files = list(directory.glob('*.csv'))
        
        if not csv_files:
            print(f"Aucun fichier CSV trouvé dans {directory_path}")
            return
        
        print(f"Fichiers CSV trouvés: {len(csv_files)}")
        
        for csv_file in sorted(csv_files):
            print(f"\n--- Traitement de {csv_file.name} ---")
            file_data = self.parse_csv_file(csv_file)
            
            # Fusionner les données par date
            for entry in file_data:
                date_str = entry['date']
                if date_str not in all_daily_data:
                    all_daily_data[date_str] = {
                        'date': date_str,
                        'quantite_mm': 0.0,
                        'neige_cm': 0.0,
                        'commentaire': ''
                    }
                
                # Additionner les valeurs (au cas où plusieurs fichiers auraient la même date)
                all_daily_data[date_str]['quantite_mm'] += entry['quantite_mm']
                all_daily_data[date_str]['neige_cm'] += entry['neige_cm']
            
            print(f"Entrées ajoutées: {len(file_data)}")
        
        # Convertir en liste et trier par date
        all_data = list(all_daily_data.values())
        all_data.sort(key=lambda x: x['date'])
        
        # Générer le fichier SQL
        self.generate_sql_file(all_data, output_file)
        
        print(f"\n✅ Traitement terminé!")
        print(f"📊 Total d'entrées uniques: {len(all_data)}")
        print(f"📄 Fichier SQL généré: {output_file}")
        
        # Statistiques
        pluie_only = len([d for d in all_data if d['quantite_mm'] > 0 and d['neige_cm'] == 0])
        neige_only = len([d for d in all_data if d['quantite_mm'] == 0 and d['neige_cm'] > 0])
        pluie_et_neige = len([d for d in all_data if d['quantite_mm'] > 0 and d['neige_cm'] > 0])
        
        print(f"📈 Statistiques:")
        print(f"   - Jours avec pluie seulement: {pluie_only}")
        print(f"   - Jours avec neige seulement: {neige_only}")
        print(f"   - Jours avec pluie ET neige: {pluie_et_neige}")
    
    def generate_sql_file(self, data_entries, output_file):
        """Génère le fichier SQL avec toutes les requêtes"""
        
        sql_content = []
        
        # En-tête du fichier
        sql_content.append("-- =============================================")
        sql_content.append("-- Données météorologiques - Import MySQL")
        sql_content.append(f"-- Généré automatiquement - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        sql_content.append(f"-- Nombre total d'entrées: {len(data_entries)}")
        sql_content.append("-- =============================================\n")
        
        # Création de la table
        sql_content.append("-- Création de la table (si elle n'existe pas)")
        sql_content.append("CREATE TABLE IF NOT EXISTS precipitations (")
        sql_content.append("    id INT AUTO_INCREMENT PRIMARY KEY,")
        sql_content.append("    date DATE NOT NULL,")
        sql_content.append("    quantite_mm DECIMAL(6,2) DEFAULT 0.00,")
        sql_content.append("    neige_cm DECIMAL(6,2) DEFAULT 0.00,")
        sql_content.append("    commentaire VARCHAR(255) DEFAULT '',")
        sql_content.append("    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,")
        sql_content.append("    INDEX idx_date (date)")
        sql_content.append(");\n")
        
        # Optionnel: vider la table existante
        sql_content.append("-- Décommentez la ligne suivante si vous voulez vider la table avant l'import")
        sql_content.append("-- TRUNCATE TABLE precipitations;\n")
        
        # Insertion des données
        sql_content.append("-- Insertion des données météorologiques")
        sql_content.append("INSERT INTO precipitations (date, quantite_mm, neige_cm, commentaire) VALUES")
        
        # Générer les valeurs
        values = []
        for entry in data_entries:
            value_str = f"('{entry['date']}', {entry['quantite_mm']:.2f}, {entry['neige_cm']:.2f}, '{entry['commentaire']}')"
            values.append(value_str)
        
        # Joindre avec des virgules et ajouter le point-virgule final
        sql_content.append(',\n'.join(values) + ";\n")
        
        # Requêtes utiles
        sql_content.append("\n-- =============================================")
        sql_content.append("-- Requêtes utiles pour vérifier les données")
        sql_content.append("-- =============================================\n")
        
        sql_content.append("-- Compter le nombre total d'entrées")
        sql_content.append("-- SELECT COUNT(*) as total_entries FROM precipitations;\n")
        
        sql_content.append("-- Voir les données par mois")
        sql_content.append("-- SELECT DATE_FORMAT(date, '%Y-%m') as mois, COUNT(*) as nb_jours, SUM(quantite_mm) as total_pluie_mm, SUM(neige_cm) as total_neige_cm FROM precipitations GROUP BY DATE_FORMAT(date, '%Y-%m') ORDER BY mois;\n")
        
        sql_content.append("-- Jours avec pluie ET neige")
        sql_content.append("-- SELECT date, quantite_mm, neige_cm FROM precipitations WHERE quantite_mm > 0 AND neige_cm > 0 ORDER BY date;\n")
        
        sql_content.append("-- Voir les 10 premiers enregistrements")
        sql_content.append("-- SELECT * FROM precipitations ORDER BY date LIMIT 10;\n")
        
        # Écrire le fichier
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write('\n'.join(sql_content))

# Fonction principale pour utiliser le convertisseur
def main():
    converter = MeteoCSVConverter()
    
    # Demander le répertoire à l'utilisateur
    directory = input("Entrez le chemin du répertoire contenant les fichiers CSV (ou appuyez sur Entrée pour le répertoire actuel): ").strip()
    
    if not directory:
        directory = "."
    
    # Demander le nom du fichier de sortie
    output_file = input("Nom du fichier SQL de sortie (défaut: precipitations_import.sql): ").strip()
    
    if not output_file:
        output_file = "precipitations_import.sql"
    
    # Traiter les fichiers
    converter.process_directory(directory, output_file)

if __name__ == "__main__":
    main()

Fichiers CSV trouvés: 52

--- Traitement de 1meteo00.csv ---
Traitement de 1meteo00.csv (année 2000)
En-têtes: ['', 'janvier', 'février', 'mars', 'avril', 'mai', 'juin']
Entrées ajoutées: 65

--- Traitement de 1meteo01.csv ---
Traitement de 1meteo01.csv (année 2001)
En-têtes: ['', 'janvier', 'février', 'mars', 'avril', 'mai', 'juin', '', 'neige', 'Neige février (cm)']
Entrées ajoutées: 85

--- Traitement de 1meteo02.csv ---
Traitement de 1meteo02.csv (année 2002)
En-têtes: ['', 'janvier', 'février', 'mars', 'avril', 'mai', 'juin']
Entrées ajoutées: 79

--- Traitement de 1meteo03.csv ---
Traitement de 1meteo03.csv (année 2003)
En-têtes: ['', 'janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'neige', 'Neige janvier cm']
Entrées ajoutées: 72

--- Traitement de 1meteo04.csv ---
Traitement de 1meteo04.csv (année 2004)
En-têtes: ['', 'janvier', 'février', 'mars', 'avril', 'mai', 'juin', '', 'neige', 'neige février', 'neige mars']
Entrées ajoutées: 74

--- Traitement de 1meteo05.csv ---
T